# Introduction
This notebook includes toy examples to demonstrate how to tune User Defined Functions with `flaml.tune`.

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the `notebook` option:
```bash
pip install flaml[notebook]
```

In [ ]:
%pip install flaml[notebook]
# from v0.6.6, catboost is made an optional dependency to build conda package.
# to install catboost without installing the notebook option, you can run:
# %pip install flaml[catboost]

# Basic tuning procedure
## 1. A basic tuning example

In [ ]:
'''Set a search space'''
from flaml import tune
config_search_space = {
    "x": tune.lograndint(lower=1, upper=100000),
    "y": tune.randint(lower=1, upper=100000)
}  

In [3]:
'''Write a evaluation function'''
import time
def evaluate_config(config: dict):
    """evaluate a hyperparameter configuration"""
    score = (config["x"] - 85000) ** 2 - config["x"] / config["y"]
    # usually the evaluation takes an non-neglible cost
    # and the cost could be related to certain hyperparameters
    # here we simulate this cost by calling the time.sleep() function
    # here we assume the cost is proportional to x
    faked_evaluation_cost = config["x"] / 100000
    time.sleep(faked_evaluation_cost)
    # we can return a single float as a score on the input config:
    # return score
    # or, we can return a dictionary that maps metric name to metric value:
    return {"score": score, "evaluation_cost": faked_evaluation_cost, "constraint_metric": config["x"] * config["y"]}

In [4]:
''''Performs tuning'''
# require: pip install flaml[blendsearch]
analysis = tune.run(
    evaluate_config,  # the function to evaluate a config
    config=config_search_space,  # the search space defined
    metric="score",
    mode="min",  # the optimization mode, "min" or "max"
    num_samples=-1,  # the maximal number of configs to try, -1 means infinite
    time_budget_s=10,  # the time budget in seconds
)

[flaml.tune.tune: 12-08 10:33:55] {486} INFO - Using search algorithm type.
You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-12-08 10:33:55,739] A new study created in memory with name: optuna
[flaml.tune.tune: 12-08 10:33:55] {636} INFO - trial 1 config: {'x': 3, 'y': 13184}
[flaml.tune.tune: 12-08 10:33:55] {636} INFO - trial 2 config: {'x': 6134, 'y': 2076}
[flaml.tune.tune: 12-08 10:33:55] {636} INFO - trial 3 config: {'x': 1143, 'y': 74880}
[flaml.tune.tune: 12-08 10:33:55] {636} INFO - trial 4 config: {'x': 5539, 'y': 1}
[flaml.tune.tune: 12-08 10:33:55] {636} INFO - trial 5 config: {'x': 6793, 'y': 16190}
[flaml.tune.tune: 12-08 10:33:55] {636} INFO - trial 6 config: {'x': 220, 'y': 22480}
[flaml.tune.t

In [5]:
'''Investigate results'''
print(analysis.best_result)

{'score': 138344643.26761267, 'evaluation_cost': 0.73238, 'constraint_metric': 7323726762, 'training_iteration': 0, 'config': {'x': 73238, 'y': 99999}, 'config/x': 73238, 'config/y': 99999, 'experiment_tag': 'exp', 'time_total_s': 0.7345480918884277}


print(analysis.result)

## Hierarchical search space 
Hierarchical search space is supported.

In [6]:
'''Set a hierarchical search space'''
from flaml import tune
gbtree_hp_space = {
        "booster": "gbtree",
        "n_estimators": tune.lograndint(lower=4, upper=64),
        "max_leaves": tune.lograndint(lower=4, upper=64),
        "learning_rate": tune.loguniform(lower=1 / 1024, upper=1.0),
    }
gblinear_hp_space = {
    "booster": "gblinear",
    "lambda": tune.uniform(0, 1),
    "alpha": tune.loguniform(0.0001, 1),
}

full_space = {
    "xgb_config": tune.choice([gbtree_hp_space, gblinear_hp_space]),
}

In [7]:
'''Write a evaluation function'''
import xgboost as xgb
def xgb_obj(X_train, X_test, y_train, y_test, config):
    config = config["xgb_config"]
    params = config2params(config)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    booster_type = config.get("booster")

    if booster_type == "gblinear":
        model = xgb.train(
            params,
            dtrain,
        )
    else:
        _n_estimators = params.pop("n_estimators")
        model = xgb.train(params, dtrain, _n_estimators)

    # get validation loss
    from sklearn.metrics import r2_score

    dtest = xgb.DMatrix(X_test)
    y_test_predict = model.predict(dtest)
    test_loss = 1.0 - r2_score(y_test, y_test_predict)
    return {"loss": test_loss}

def config2params(config: dict) -> dict:
    params = config.copy()
    max_depth = params["max_depth"] = params.get("max_depth", 0)
    if max_depth == 0:
        params["grow_policy"] = params.get("grow_policy", "lossguide")
        params["tree_method"] = params.get("tree_method", "hist")
    # params["booster"] = params.get("booster", "gbtree")
    params["use_label_encoder"] = params.get("use_label_encoder", False)
    if "n_jobs" in config:
        params["nthread"] = params.pop("n_jobs")
    return params

In [ ]:
'''Tune xgb_obj with configs from the hierarchical search space'''
from flaml.data import load_openml_dataset
from functools import partial

X_train, X_test, y_train, y_test = load_openml_dataset(
    dataset_id=537, data_dir="./"
)
analysis = tune.run(
    partial(xgb_obj, X_train, X_test, y_train, y_test),
    config=full_space,
    metric="loss",
    mode="min",
    num_samples=5,
)
print("analysis", analysis.results)

# Advanced Tuning Options

## 1. Constraints on the tuning

1. A user can specify constraints on the configurations to be satisfied via the argument `config_constraints`.

In [9]:
from flaml import tune
def area(config):
    return config["width"] * config["length"]

cube_search_space = {
    "width": tune.lograndint(lower=1, upper=1000),
    "height": tune.randint(lower=1, upper=1000),
    "length": tune.randint(lower=1, upper=1000),
}

def cube_volume(config: dict):
    """evaluate a hyperparameter configuration"""
    score = config["width"] * config["height"] * config["length"]
    return {"volume": score}

analysis = tune.run(evaluation_function=cube_volume,
         mode="min",
         metric="volume",
         config=cube_search_space,
         config_constraints=[(area, "<=", 1000)],
         num_samples=5,
        )
print(analysis.best_result)

[flaml.tune.tune: 12-08 10:34:07] {486} INFO - Using search algorithm type.
You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-12-08 10:34:07,631] A new study created in memory with name: optuna
[flaml.tune.tune: 12-08 10:34:07] {636} INFO - trial 1 config: {'width': 1, 'height': 132, 'length': 647}
[flaml.tune.tune: 12-08 10:34:07] {636} INFO - trial 2 config: {'width': 2, 'height': 760, 'length': 169}
[flaml.tune.tune: 12-08 10:34:07] {636} INFO - trial 3 config: {'width': 1, 'height': 685, 'length': 953}
[flaml.tune.tune: 12-08 10:34:07] {636} INFO - trial 4 config: {'width': 1, 'height': 512, 'length': 812}
[flaml.tune.tune: 12-08 10:34:07] {636} INFO - trial 5 config: {'width': 1, 'height': 373, 'length': 6

{'volume': 85404, 'training_iteration': 0, 'config': {'width': 1, 'height': 132, 'length': 647}, 'config/width': 1, 'config/height': 132, 'config/length': 647, 'experiment_tag': 'exp', 'time_total_s': 0.0012857913970947266}


2.  You can also specify a list of metric constraints to be satisfied via the argument `metric_constraints`.

In [10]:
from flaml import tune
tune.run(evaluation_function=evaluate_config,
         mode="min",
         metric="score",
         config=config_search_space,
         metric_constraints=[("evaluation_cost", "<=", 0.1)],
         num_samples=5,)

[flaml.tune.tune: 12-08 10:34:07] {486} INFO - Using search algorithm type.
You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-12-08 10:34:07,831] A new study created in memory with name: optuna
[flaml.tune.tune: 12-08 10:34:07] {636} INFO - trial 1 config: {'x': 3, 'y': 13184}
[flaml.tune.tune: 12-08 10:34:07] {636} INFO - trial 2 config: {'x': 6134, 'y': 2076}
[flaml.tune.tune: 12-08 10:34:07] {636} INFO - trial 3 config: {'x': 1143, 'y': 74880}
[flaml.tune.tune: 12-08 10:34:07] {636} INFO - trial 4 config: {'x': 5539, 'y': 1}
[flaml.tune.tune: 12-08 10:34:07] {636} INFO - trial 5 config: {'x': 6793, 'y': 16190}


### config_constraints vs metric_constraints

In [11]:
'''Write a evaluation function'''
import xgboost as xgb
from flaml import tune
import time
def xgb_simple_obj(X_train, X_test, y_train, y_test, config):
    params = config2params(config)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    start_time = time.time()
    _n_estimators = params.pop("n_estimators")
    model = xgb.train(params, dtrain, _n_estimators)
    end_time = time.time()
    # get validation loss
    from sklearn.metrics import r2_score
    dtest = xgb.DMatrix(X_test)
    y_test_predict = model.predict(dtest)
    test_loss = 1.0 - r2_score(y_test, y_test_predict)
    return {"loss": test_loss, "training_cost": end_time-start_time}

def config2params(config: dict) -> dict:
    params = config.copy()
    max_depth = params["max_depth"] = params.get("max_depth", 0)
    if max_depth == 0:
        params["grow_policy"] = params.get("grow_policy", "lossguide")
        params["tree_method"] = params.get("tree_method", "hist")
    # params["booster"] = params.get("booster", "gbtree")
    params["use_label_encoder"] = params.get("use_label_encoder", False)
    if "n_jobs" in config:
        params["nthread"] = params.pop("n_jobs")
    return params

def my_model_size(config):
    return config["n_estimators"] * config["max_leaves"]

'''Tune xgb_obj with configs from the hierarchical search space'''
from flaml.data import load_openml_dataset
from functools import partial

xgb_space = {
     "n_estimators": tune.randint(lower=4, upper=64),
      "max_leaves": tune.randint(lower=4, upper=64),
      "learning_rate": tune.loguniform(lower=1 / 1024, upper=1.0),
}
X_train, X_test, y_train, y_test = load_openml_dataset(
    dataset_id=537, data_dir="./"
)
analysis = tune.run(
    partial(xgb_simple_obj, X_train, X_test, y_train, y_test),
    config=xgb_space,
    metric="loss",
    mode="min",
    config_constraints = [(my_model_size, "<=", 40)],
    metric_constraints = [("training_cost", "<=", 1)],
    num_samples=20,
)
print("analysis", analysis.results)

[flaml.tune.tune: 12-08 10:34:08] {486} INFO - Using search algorithm type.
You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-12-08 10:34:08,193] A new study created in memory with name: optuna
[flaml.tune.tune: 12-08 10:34:08] {636} INFO - trial 1 config: {'n_estimators': 7, 'max_leaves': 5, 'learning_rate': 0.6111947006764871}


load dataset from ./openml_ds537.pkl
Dataset name: houses
X_train.shape: (15480, 8), y_train.shape: (15480,);
X_test.shape: (5160, 8), y_test.shape: (5160,)
[10:34:08] WARNING: ../src/learner.cc:541: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/home/qxw5138/miniconda3/envs/myflaml/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[flaml.tune.tune: 12-08 10:34:08] {636} INFO - trial 2 config: {'n_estimators': 6, 'max_leaves': 5, 'learning_rate': 0.18074443349590638}
/home/qxw5138/miniconda3/envs/myflaml/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
Received additional result for trial e6bcd752, but it already finished. Result: {'loss': 0.8238569381974518, 'training_cost': 0.021718502044677734, 'training_iteration': 0, 'config': {'n_estimators': 6, 'max_leaves': 5, 'learning_rate': 0.18074443349590638}, 'config/n_estimators': 6, 'config/m

[10:34:08] WARNING: ../src/learner.cc:541: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/home/qxw5138/miniconda3/envs/myflaml/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[flaml.tune.tune: 12-08 10:34:09] {636} INFO - trial 4 config: {'n_estimators': 4, 'max_leaves': 8, 'learning_rate': 0.24184523333348865}
/home/qxw5138/miniconda3/envs/myflaml/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


...

[10:34:10] WARNING: ../src/learner.cc:541: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:34:10] WARNING: ../src/learner.cc:541: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:34:10] WARNING: ../src/learner.cc:541: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find abov

[flaml.tune.tune: 12-08 10:34:10] {636} INFO - trial 16 config: {'n_estimators': 8, 'max_leaves': 4, 'learning_rate': 0.005400819560603306}
/home/qxw5138/miniconda3/envs/myflaml/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[flaml.tune.tune: 12-08 10:34:10] {636} INFO - trial 17 config: {'n_estimators': 8, 'max_leaves': 4, 'learning_rate': 0.0056842152617478025}


[10:34:10] WARNING: ../src/learner.cc:541: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:34:10] WARNING: ../src/learner.cc:541: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/home/qxw5138/miniconda3/envs/myflaml/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[flaml.tune.tune: 12-08 10:34:10] {636} INFO - trial 18 config: {'n_estimators': 8, 'max_leaves': 4, 'learning_rate': 0.005370232296413302}
/home/qxw5138/miniconda3/envs/myflaml/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[flaml.tune.tune: 12-08 10:34:10] {636} INFO - trial 19 config: {'n_estimators': 8, 'max_leaves': 4, 'learning_rate': 0.005273310814672741}
/home/qxw5138/miniconda3/envs/myflaml/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be remove

[10:34:10] WARNING: ../src/learner.cc:541: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:34:10] WARNING: ../src/learner.cc:541: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:34:10] WARNING: ../src/learner.cc:541: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find abov

## Parallel tuning

In [12]:
%pip install flaml[ray]

Note: you may need to restart the kernel to use updated packages.


In [13]:
# require: pip install flaml[ray]
analysis = tune.run(
    evaluate_config,  # the function to evaluate a config
    config=config_search_space,  # the search space defined
    metric="score",
    mode="min",  # the optimization mode, "min" or "max"
    num_samples=-1,  # the maximal number of configs to try, -1 means infinite
    time_budget_s=10,  # the time budget in seconds
    use_ray=True,
    resources_per_trial={"cpu": 2}  # limit resources allocated per trial
)
print(analysis.best_trial.last_result)  # the best trial's result
print(analysis.best_config)  # the best config

You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-12-08 10:34:13,894] A new study created in memory with name: optuna
2022-12-08 10:34:17,455	WARNING function_runner.py:603 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-12-08 10:34:19,727	WARNING tune.py:668 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial evaluate_config_ed9cac64 reported score=7224490009.00 with parameters={'x': 3, 'y': 13184}.
Trial evaluate_config_ed9cac64 completed. Last result: score=7224490008.999772,evaluation_cost=3e-05,constraint_metric=39552
Trial evaluate_config_eeb6493f reported score=7031996448.98 with parameters={'x': 1143, 'y': 74880}.
Trial evaluate_config_eeb6493f completed. Last result: score=7031996448.9847355,evaluation_cost=0.01143,constraint_metric=85587840
Trial evaluate_config_eeb64941 reported score=7223980036.00 with parameters={'x': 6, 'y': 76053}.
Trial evaluate_config_eeb64941 completed. Last result: score=7223980035.999921,evaluation_cost=6e-05,constraint_metric=456318
Trial evaluate_config_eeb64943 reported score=6864453903.98 with parameters={'x': 2148, 'y': 95339}.
Trial evaluate_config_eeb64943 completed. Last result: score=6864453903.977469,evaluation_cost=0.02148,constraint_metric=204788172
Trial evaluate_config_eeb64946 reported score=6303883608.92 with parameters={'x': 5603, '

Trial evaluate_config_efa2fe5d reported score=5781321224.90 with parameters={'x': 8965, 'y': 91479}.
Trial evaluate_config_efa2fe4b reported score=16908541.34 with parameters={'x': 89112, 'y': 33454}.
Trial evaluate_config_efa2fe57 completed. Last result: score=3746786520.588504,evaluation_cost=0.23789,constraint_metric=1375265879
Trial evaluate_config_efa2fe4b completed. Last result: score=16908541.336282656,evaluation_cost=0.89112,constraint_metric=2981152848
Trial evaluate_config_efa2fe5d completed. Last result: score=5781321224.901999,evaluation_cost=0.08965,constraint_metric=820109235
Trial evaluate_config_efa2fe5f reported score=1929405624.41 with parameters={'x': 41075, 'y': 70084}.
Trial evaluate_config_eeb6495f completed. Last result: score=7224660003.999976,evaluation_cost=2e-05,constraint_metric=165736
Trial evaluate_config_efa2fe5f completed. Last result: score=1929405624.4139175,evaluation_cost=0.41075,constraint_metric=2878700300
Trial evaluate_config_efa2fe54 reported sc

2022-12-08 10:34:30,080	INFO stopper.py:363 -- Reached timeout of 10 seconds. Stopping all trials.


Trial evaluate_config_f1f80183 reported score=955366279.22 with parameters={'x': 54091, 'y': 30467}.
Trial evaluate_config_f1f80180 completed. Last result: score=1103568398.4442508,evaluation_cost=0.5178,constraint_metric=1723393740
Trial evaluate_config_f32cfd2e reported score=466344023.17 with parameters={'x': 63405, 'y': 34639}.


Trial name,status,loc,x,y,iter,total time (s),score,evaluation_cost,constraint_metric
evaluate_config_ed9cac64,TERMINATED,130.203.136.143:2091440,3,13184,1,0.000496149,7.22449e+09,3e-05,39552
evaluate_config_eeb6493e,TERMINATED,130.203.136.143:2091474,6134,2076,1,0.061902,6.21985e+09,0.06134,12734184
evaluate_config_eeb6493f,TERMINATED,130.203.136.143:2091440,1143,74880,1,0.0118706,7.032e+09,0.01143,85587840
evaluate_config_eeb64940,TERMINATED,130.203.136.143:2091480,220,22480,1,0.00257039,7.18765e+09,0.0022,4945600
evaluate_config_eeb64941,TERMINATED,130.203.136.143:2091440,6,76053,1,0.00046587,7.22398e+09,6e-05,456318
evaluate_config_eeb64942,TERMINATED,130.203.136.143:2091484,4,8834,1,0.000368118,7.22432e+09,4e-05,35336
evaluate_config_eeb64943,TERMINATED,130.203.136.143:2091440,2148,95339,1,0.0218968,6.86445e+09,0.02148,204788172
evaluate_config_eeb64944,TERMINATED,130.203.136.143:2091491,1032,60766,1,0.0106943,7.05063e+09,0.01032,62710512
evaluate_config_eeb64945,TERMINATED,130.203.136.143:2091494,1266,88994,1,0.0130966,7.01138e+09,0.01266,112666404
evaluate_config_eeb64946,TERMINATED,130.203.136.143:2091440,5603,71825,1,0.056505,6.30388e+09,0.05603,402435475


2022-12-08 10:34:31,049	INFO tune.py:747 -- Total run time: 13.60 seconds (10.62 seconds for the tuning loop).


{'score': 25280.148097198515, 'evaluation_cost': 0.84841, 'constraint_metric': 8449315190, 'time_this_iter_s': 0.8496530055999756, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 1, 'trial_id': 'f0c474c9', 'experiment_id': '78aa09b8180c42ebba808cfd6754a5b7', 'date': '2022-12-08_10-34-26', 'timestamp': 1670524466, 'time_total_s': 0.8496530055999756, 'pid': 2091499, 'hostname': 'i4-l-qxw5138-01.ad.psu.edu', 'node_ip': '130.203.136.143', 'config': {'x': 84841, 'y': 99590}, 'time_since_restore': 0.8496530055999756, 'timesteps_since_restore': 0, 'iterations_since_restore': 1, 'warmup_time': 0.005667686462402344, 'experiment_tag': '90_x=84841,y=99590'}
{'x': 84841, 'y': 99590}


## Warm start

In [14]:
from flaml import tune

config_search_space = {
    "a": tune.uniform(lower=0, upper=0.99),
    "b": tune.uniform(lower=0, upper=3),
}

def simple_obj(config):
    return config["a"] + config["b"]

points_to_evaluate = [
    {"b": .99, "a": 3},
    {"b": .99, "a": 2},
    {"b": .80, "a": 3},
    {"b": .80, "a": 2},
]
evaluated_rewards = [3.99, 2.99]

analysis = tune.run(
    simple_obj,
    config=config_search_space,
    mode="max",
    points_to_evaluate=points_to_evaluate,
    evaluated_rewards=evaluated_rewards,
    num_samples=10,
)

[flaml.tune.tune: 12-08 10:34:31] {486} INFO - Using search algorithm type.
2022-12-08 10:34:31,988	WARNING optuna.py:297 -- You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-12-08 10:34:31,991] A new study created in memory with name: optuna
/home/qxw5138/miniconda3/envs/myflaml/lib/python3.8/site-packages/ray/tune/suggest/optuna.py:561: ExperimentalWarning: create_trial is experimental (supported from v2.0.0). The interface can change in the future.
  trial = ot.trial.create_trial(
2022-12-08 10:34:31,994	WARNING optuna.py:297 -- You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces onl

## Trial scheduling

###  An authentic scheduler implemented in FLAML (`scheduler='flaml'`).

In [15]:
search_space = {
        "n_estimators": tune.lograndint(lower=4, upper=32768),
        "max_leaves": tune.lograndint(lower=4, upper=32768),
        "learning_rate": tune.loguniform(lower=1 / 1024, upper=1.0),
    }

In [16]:
'''Set a evaluation function with resource dimension'''
def obj_from_resource_attr(resource_attr, X_train, X_test, y_train, y_test, config):
    from lightgbm import LGBMClassifier
    from sklearn.metrics import accuracy_score

    # in this example sample size is our resource dimension
    resource = int(config[resource_attr])
    sampled_X_train = X_train.iloc[:resource]
    sampled_y_train = y_train[:resource]

    # construct a LGBM model from the config
    # note that you need to first remove the resource_attr field
    # from the config as it is not part of the original search space
    model_config = config.copy()
    del model_config[resource_attr]
    model = LGBMClassifier(**model_config)

    model.fit(sampled_X_train, sampled_y_train)
    y_test_predict = model.predict(X_test)
    test_loss = 1.0 - accuracy_score(y_test, y_test_predict)
    return {"loss": test_loss}

In [17]:
from flaml import tune
from functools import partial
from flaml.data import load_openml_task
    
X_train, X_test, y_train, y_test = load_openml_task(task_id=7592, data_dir="")
max_resource = len(y_train)
resource_attr = "sample_size"
min_resource = 1000
analysis = tune.run(
    partial(
        obj_from_resource_attr, resource_attr, X_train, X_test, y_train, y_test
    ),
    config=search_space,
    metric="loss",
    mode="min",
    resource_attr=resource_attr,
    scheduler="flaml",
    max_resource=max_resource,
    min_resource=min_resource,
    reduction_factor=2,
    time_budget_s=10,
    num_samples=-1,
)
print("best result w/ flaml scheduler (in 10s): ", analysis.best_result)

[flaml.tune.tune: 12-08 10:34:32] {486} INFO - Using search algorithm type.
2022-12-08 10:34:32,611	WARNING optuna.py:297 -- You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-12-08 10:34:32,612] A new study created in memory with name: optuna
[flaml.tune.tune: 12-08 10:34:32] {636} INFO - trial 1 config: {'n_estimators': 9, 'max_leaves': 1364, 'learning_rate': 0.012074374674294664, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:32] {636} INFO - trial 2 config: {'n_estimators': 4048, 'max_leaves': 4, 'learning_rate': 0.07891713267442702, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:32] {636} INFO - trial 3 config: {'n_estimators': 3295, 'max_leaves': 334, 'learning_rate': 0.004638797085780012, 'sample_

load dataset from openml_task7592.pkl
X_train.shape: (43957, 14), y_train.shape: (43957,),
X_test.shape: (4885, 14), y_test.shape: (4885,)
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=334 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3668 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1845 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=379 will be ignored. Current value: num_leaves=31


[flaml.tune.tune: 12-08 10:34:32] {636} INFO - trial 7 config: {'n_estimators': 948, 'max_leaves': 2573, 'learning_rate': 0.0073847289359894605, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:32] {636} INFO - trial 8 config: {'n_estimators': 15449, 'max_leaves': 2409, 'learning_rate': 0.04196829547317673, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:32] {636} INFO - trial 9 config: {'n_estimators': 13, 'max_leaves': 106, 'learning_rate': 0.10448271169801053, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:32] {636} INFO - trial 10 config: {'n_estimators': 199, 'max_leaves': 185, 'learning_rate': 0.07069097177435173, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:32] {636} INFO - trial 11 config: {'n_estimators': 382, 'max_leaves': 1340, 'learning_rate': 0.06295171682621144, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:33] {636} INFO - trial 12 config: {'n_estimators': 5520, 'max_leaves': 413, 'learning_rate': 0.5308914484951052, 'sample_size': 1000}


[LightGBM] [Warning] num_leaves is set=31, max_leaves=2573 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=2409 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=106 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=185 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1340 will be ignored. Current value: num_leaves=31


[flaml.tune.tune: 12-08 10:34:33] {636} INFO - trial 13 config: {'n_estimators': 3503, 'max_leaves': 7, 'learning_rate': 0.0010918443163764437, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:33] {636} INFO - trial 14 config: {'n_estimators': 54, 'max_leaves': 30204, 'learning_rate': 0.004360679238440181, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:33] {636} INFO - trial 15 config: {'n_estimators': 49, 'max_leaves': 23677, 'learning_rate': 0.0010641529214071718, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:33] {636} INFO - trial 16 config: {'n_estimators': 1534, 'max_leaves': 37, 'learning_rate': 0.006371183957388993, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:33] {636} INFO - trial 17 config: {'n_estimators': 29723, 'max_leaves': 9340, 'learning_rate': 0.0027991149620651954, 'sample_size': 1000}


...

[LightGBM] [Warning] num_leaves is set=31, max_leaves=1016 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1863 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1124 will be ignored. Current value: num_leaves=31


[flaml.tune.tune: 12-08 10:34:42] {636} INFO - trial 179 config: {'n_estimators': 15077, 'max_leaves': 4, 'learning_rate': 0.001116720666995303, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:42] {636} INFO - trial 180 config: {'n_estimators': 25744, 'max_leaves': 3066, 'learning_rate': 0.13462790568725672, 'sample_size': 1000}


[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3066 will be ignored. Current value: num_leaves=31


[flaml.tune.tune: 12-08 10:34:42] {636} INFO - trial 181 config: {'n_estimators': 14379, 'max_leaves': 21532, 'learning_rate': 0.11249756313060533, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:42] {636} INFO - trial 182 config: {'n_estimators': 43, 'max_leaves': 6487, 'learning_rate': 0.17296950883370182, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:42] {636} INFO - trial 183 config: {'n_estimators': 22, 'max_leaves': 7280, 'learning_rate': 0.258852421157622, 'sample_size': 1000}
[flaml.tune.tune: 12-08 10:34:42] {636} INFO - trial 184 config: {'n_estimators': 45, 'max_leaves': 3571, 'learning_rate': 0.08732512289838663, 'sample_size': 1000}


[LightGBM] [Warning] num_leaves is set=31, max_leaves=21532 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=6487 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=7280 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3571 will be ignored. Current value: num_leaves=31
best result w/ flaml scheduler (in 10s):  {'loss': 0.2393039918116684, 'training_iteration': 0, 'config': {'n_estimators': 9, 'max_leaves': 1364, 'learning_rate': 0.012074374674294664, 'sample_size': 1000}, 'config/n_estimators': 9, 'config/max_leaves': 1364, 'config/learning_rate': 0.012074374674294664, 'config/sample_size': 1000, 'experiment_tag': 'exp', 'time_total_s': 0.025426626205444336}


###  ASHA scheduler (`scheduler='asha'`) or a custom scheduler of the  [`TrialScheduler`](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers) class from `ray.tune`.

In [18]:
def obj_w_intermediate_report(
        resource_attr,
        X_train,
        X_test,
        y_train,
        y_test,
        min_resource,
        max_resource,
        config,
    ):
    from lightgbm import LGBMClassifier
    from sklearn.metrics import accuracy_score

    # a customized schedule to perform the evaluation
    eval_schedule = [res for res in range(min_resource, max_resource, 5000)] + [
        max_resource
    ]
    for resource in eval_schedule:
        sampled_X_train = X_train.iloc[:resource]
        sampled_y_train = y_train[:resource]

        # construct a LGBM model from the config
        model = LGBMClassifier(**config)

        model.fit(sampled_X_train, sampled_y_train)
        y_test_predict = model.predict(X_test)
        test_loss = 1.0 - accuracy_score(y_test, y_test_predict)
        # need to report the resource attribute used and the corresponding intermediate results
        try:
            tune.report(sample_size=resource, loss=test_loss)
        except StopIteration:
            return

In [19]:
X_train, X_test, y_train, y_test = load_openml_task(task_id=7592, data_dir="")
resource_attr = "sample_size"
min_resource = 1000
max_resource = len(y_train)
analysis = tune.run(
    partial(
        obj_w_intermediate_report,
        resource_attr,
        X_train,
        X_test,
        y_train,
        y_test,
        min_resource,
        max_resource,
    ),
    config=search_space,
    metric="loss",
    mode="min",
    resource_attr=resource_attr,
    scheduler="asha",
    max_resource=max_resource,
    min_resource=min_resource,
    reduction_factor=2,
    time_budget_s=10,
    num_samples=-1,
)
print("best result w/ asha scheduler (in 10s): ", analysis.best_result)

[flaml.tune.tune: 12-08 10:34:42] {486} INFO - Using search algorithm type.
2022-12-08 10:34:42,987	WARNING optuna.py:297 -- You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-12-08 10:34:42,988] A new study created in memory with name: optuna
[flaml.tune.tune: 12-08 10:34:42] {636} INFO - trial 1 config: {'n_estimators': 9, 'max_leaves': 1364, 'learning_rate': 0.012074374674294664}


load dataset from openml_task7592.pkl
X_train.shape: (43957, 14), y_train.shape: (43957,),
X_test.shape: (4885, 14), y_test.shape: (4885,)
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] 

[flaml.tune.tune: 12-08 10:34:43] {636} INFO - trial 2 config: {'n_estimators': 4048, 'max_leaves': 4, 'learning_rate': 0.07891713267442702}


[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current v